In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

#from ipynb.fs.full.proj1_SA import clean_convert # This is the function that cleans the textfile and 
    

In [2]:
# This utility was created because students were getting confused when they ran 
# their notebooks twice, the previous write-to-file code would do nothing and say  
# nothing. The students thought the file was over-written when in fact, it was not -
# generating hidden bugs in subsequent notebooks.

def save_file(data, fname, dname):
    """Save a datafile (data) to a specific location (dname) and filename (fname)
    
    Currently valid formats are limited to CSV or PKL."""
    
    if not os.path.exists(dname):
        os.mkdir(dname)
        print(f'Directory {dname} was created.')
        
    fpath = os.path.join(dname, fname)
    
    
    if os.path.exists(fpath):
        print("A file already exists with this name.\n")

        yesno = None
        while yesno != "Y" and yesno != "N":
            yesno = input('Do you want to overwrite? (Y/N)').strip()[0].capitalize()
            if yesno == "Y":
                print(f'Writing file.  "{fpath}"')
                _save_file(data, fpath)
                break  # Not required
            elif yesno == "N":
                print('\nPlease re-run this cell with a new filename.')
                break  # Not required
            else:
                print('\nUnknown input, please enter "Y" or "N".')

    else:  # path does not exist, ok to save the file
        print(f'Writing file.  "{fpath}"')
        _save_file(data, fpath)
        
        
        
        
        
        
def _save_file(data, fpath):
    valid_ftypes = ['.csv', '.pkl']
    
    assert (fpath[-4:] in valid_ftypes), "Invalid file type.  Use '.csv' or '.pkl'"

    # Figure out what kind of file we're dealing with by name
    if fpath[-3:] == 'csv':
        data.to_csv(fpath, index=False)
    elif fpath[-3:] == 'pkl':
        with open(fpath, 'wb') as f:
            pickle.dump(data, f)
            

In [3]:
def extract_icd_code(df):
    dict0={}
    for member in df['member_id'].unique():
        lst=[]
        for item in df[df['member_id']==member]['icd_code'].unique():
            if item: # if not none
                lst.append(str(item).split(';')) # combine all the values in icd_code for each member_id
        list_icd_codes=[]
        for item in lst:
            for i in range(len(item)):
                list_icd_codes.append(item[i])
#            
        dict0[member]=list(set(list_icd_codes).difference({'nan'}))  
    return dict0



In [4]:
# extract_icd_code(df1)

In [5]:
def df1_one_record_per_member_id(Df):
#
    df=pd.DataFrame({}) 
    df['member_id']=Df['member_id'].unique() # create a dataframe to store the member_id
                                              # one record per member_id                    
#
    df['num_of_visit']=0 # create one column to store the total number of 
                         # hospital visits per member id 
#    
    for item in range(20):
#
        df['v'+str(item+1)]=0 # create 41 columns for the visit   
                              # number (first visit , second visit etc.)
    #
        df['v'+str(item+1)+'_'+'re_adm']=0 # create 41 columns for re_admission
    #                                      per visit (v1_re_adm, v2_re_adm, ...etc.)
    #
        df['v'+str(item+1)+'_'+'er_to_inp']=0 # create 41 columns for er_to_inp_admission_transfer
    #                                        per visit (v1_er_to_inp, v2_er_to_inp, ...etc)
    #
        df['v'+str(item+1)+'_'+'days_to_prev']=0 # create 41 columns for days_to_prev_admission
    #                                             per visit (v1_days_to_prev, v2_days__to_prev,...etc)
    #
        df['v'+str(item+1)+'_'+'len']=0 # create 41 columns for visit_duration
    #                                          per visit (v1_len, v2_len, ... etc)
    #
        df['v'+str(item+1)+'_'+'len_pos']=0 # create 41 columns for visit_duration_updated
    #                                         per visit (v1_len_pos, v2_len_pos,...etc.)
    #
        for j in Df['admission_type'].unique(): # create 41x9 columns per visit per admission type
                                                 # (v1_OTH, v1_ER,..., v2_OTH, v2_ER...etc.)
            df['v'+str(item+1)+'_'+str(j)]=0     
#        
    return(df)            

In [6]:
def fill_df1_res(df1_res,df1): # df1 will be df1_subset when we use this function.
#    
    for mem_id in df1_res['member_id']:    #    
        num=df1[df1['member_id']==mem_id]['nth_visit'].max()
        df1_res.loc[df1_res['member_id']==mem_id, 'num_of_visit']=num  
#           
        for item in range(num):
            df1_res.loc[df1_res['member_id']== mem_id, 'v'+str(item+1)]=1                                 
            #
            if 'Yes' in df1[(df1['member_id']==mem_id) & (df1['nth_visit']==item+1)]['is_readmission'].unique():
            #          
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'re_adm']=1            
            #
            if 'Y' in df1[(df1['member_id']==mem_id) & (df1['nth_visit']==item+1)]['er_to_inp_admission_transfer'].unique():
            #          
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'er_to_inp']=1            
            #          
            for j in df1[(df1['member_id']==mem_id) & (df1['nth_visit']==item+1)]['admission_type']:
            #          
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+str(j)]=1
    #
            for value in df1[(df1['member_id']==mem_id)&(df1['nth_visit']==item+1)]['days_to_prev_admission']:
            #
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'days_to_prev']=value
            #
            for value in df1[(df1['member_id']==mem_id)&(df1['nth_visit']==item+1)]['days_to_prev_admission']:
            #
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'days_to_prev']=value
            #
            for value in df1[(df1['member_id']==mem_id)&(df1['nth_visit']==item+1)]['visit_duration']:
        #
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'len']=value
        #
                df1_res.loc[df1_res['member_id']==mem_id, 'v'+str(item+1)+'_'+'len_pos']=value+1
         #   
    return(df1_res)

In [7]:
def df2_one_record_per_member_id(df): # df will be df2_subset when we use this function.
    df2_res=pd.DataFrame({}) 
    df2_res['member_id']=df['member_id'].unique() # create a dataframe to store the member_id
                                              # one record per member_id in df2                   
    df2_res['icd_'+str('I10')]=0 # create one column per icd code   
    # 
    df2_res['icd_'+str('I10')+'_'+'chronic']=0 # create one column for each icd code to store 
                                                  # to store whether the ilness is chronic or unknown
    df2_res['icd_'+str('I739')]=0 # create one column per icd code   
    # 
    df2_res['icd_'+str('I739')+'_'+'chronic']=0 # create one column for each icd code to store 
                                                  # to store whether the ilness is chronic or unknown
    return(df2_res)            

In [8]:
def fill_df2_res(df2_res, df2_subset):
    #
    for mem_id in df2_res['member_id'].unique():
#    
        for item in df2_subset[df2_subset['member_id']==mem_id]['icd_code']:
            #
            df2_res.loc[df2_res['member_id']== mem_id, 'icd_'+str(item)]=1                                 
            #        
            if 'Chronic' in df2_subset[(df2_subset['member_id']==mem_id) & (df2_subset['icd_code']==item)]['icd_chronic_or_acute'].unique():
            #          
                df2_res.loc[df2_res['member_id']==mem_id, 'icd_'+str(item)+'_'+'chronic']=1            
    return df2_res
        

In [9]:
def df3_one_record_per_member_id(df): # df will be df3_subset when we use this function
#
    df3_res=pd.DataFrame({}) 
    df3_res['member_id']=df['member_id'].unique() # create a dataframe to store the member_id
                                              # one record per member_id in df2                   
    #    
    for item in df['loinc_code'].unique():
        #
        df3_res['lab_'+str(item)]=0 # create one column per icd code   
         #                                
        df3_res['lab_'+str(item)+'_'+'res']=0 # create one column for each icd code to store 
                                                  # to store whether the ilness is chronic or unknown   
    return(df3_res)
            

In [10]:
def fill_df3_res(df3_res, df3): # df3 will bbe df3_subset when we use this function.
    #
    for mem_id in df3_res['member_id'].unique():
        
        for item in df3[df3['member_id']==mem_id]['loinc_code'].unique():
            #              
            df3_res.loc[df3_res['member_id']==mem_id, 'lab_'+str(item)]=1            
            #
            for value in df3[(df3['member_id']==mem_id)&(df3['loinc_code']==item)]['abnormal_code_updated']:
                #    
                df3_res.loc[df3_res['member_id']==mem_id, 'lab_'+str(item)+'_'+'res']=value
    return df3_res

In [11]:
def df5_one_record_per_member_id(df): # df will be df5_subset when we use this function
#
    df5_res=pd.DataFrame({}) 
    df5_res['member_id']=df['member_id'].unique() # create a dataframe to store the member_id
                                              # one record per member_id in df2                   
    #    
    for item in df['ndc_number'].unique():
        #
        df5_res['ndc#_'+str(item)]=0 # create one column per icd code   
         #                                
        df5_res['ndc#_'+str(item)+'_'+'rate']=0 # create one column for each icd code to store 
                                                  # to store whether the ilness is chronic or unknown 
        df5_res['ncd#_'+str(item)+'_'+'refill']=0    
    return df5_res
            

In [12]:
def fill_df5_res(df5_res, df5): # df5 will be df5_subset when we use this function.
    #
    for mem_id in df5_res['member_id'].unique():
        #
        for item in df5[df5['member_id']==mem_id]['ndc_number'].unique():
            #              
            df5_res.loc[df5_res['member_id']==mem_id, 'ndc#_'+str(item)]=1            
            #
            for value in df5[(df5['member_id']==mem_id)&(df5['ndc_number']==item)]['drug_quantity_rate']:
                #    
                df5_res.loc[df5_res['member_id']==mem_id, 'ndc#_'+str(item)+'_'+'rate']=value
                #
            if 'R' in df5[(df5['member_id']==mem_id) & (df5['ndc_number']==item)]['new_or_refill'].unique():
            #          
                df5_res.loc[df5_res['member_id']==mem_id, 'ndc#_'+str(item)+'_'+'refill']=1            
            
    return df5_res